# IRGAN: Generative Adversarial Nets for Information Retrival

This notebook provides the implimentations of [IRGAN](https://arxiv.org/pdf/1705.10513.pdf) for a learning to rank of a pairwise approach published on SIGIR 2017.

Copyright (C) 2017 Takahiro Ishikawa  
  
Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:  

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.  
  
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [1]:
import numpy as np
import tensorflow as tf

## Dataset (MQ2008-semi)

You should download the data set from [here](https://drive.google.com/drive/folders/0B-dulzPp3MmCM01kYlhhNGQ0djA?usp=sharing), and then put in MQ2008-semi/.

In [2]:
class MQ2008NoTargetException(Exception):
    pass

class MQ2008:
    def __init__(self, dataset_dir='MQ2008-semi'):
        self.pool = {}
        self.pool['train'] = self._load_data(dataset_dir + '/train.txt')
        self.pool['test'] = self._load_data(dataset_dir + '/test.txt')
        
    def get_queries(self, target='train'):
        if target in self.pool.keys():
            return list(self.pool[target].keys())
        else:
            raise MQ2008NoTargetException()

    def get_docs(self, query, target='train'):
        if target in self.pool.keys():
            return list(self.pool[target][query].keys())
        else:
            raise MQ2008NoTargetException()
    
    def get_features(self, query, doc, target='train'):
        if target in self.pool.keys():
            return self.pool[target][query][doc]['f']
        else:
            raise MQ2008NoTargetException()
    
    def get_rank(self, query, doc, target='train'):
        if target in self.pool.keys():
            return self.pool[target][query][doc]['r']
        else:
            raise MQ2008NoTargetException()
 
    def get_pos_queries(self, target='train'):
        if target in self.pool.keys():
            return list({query for query in self.get_queries(target=target)
                         for doc in self.get_docs(query, target=target) if self.get_rank(query, doc, target=target) > 0.0})
        else:
            raise MQ2008NoTargetException()
            
    def get_pos_docs(self, query, target='train'):
        if target in self.pool.keys():
            return list({doc for doc in self.get_docs(query, target=target) if self.get_rank(query, doc, target=target) > 0.0})
        else:
            raise MQ2008NoTargetException()

    # load docs and features for a query.
    def _load_data(self, file, feature_size=46):
        query_doc_feature = {}
        with open(file) as f:
            for line in f:
                cols = line.strip().split()
                rank = cols[0]
                query = cols[1].split(':')[1]
                doc = cols[-7]
                feature = []
                for i in range(2, 2 + feature_size):
                    feature.append(float(cols[i].split(':')[1]))
                if query in query_doc_feature.keys():
                    query_doc_feature[query][doc] = {'r': float(rank), 'f': np.array(feature)}
                else:
                    query_doc_feature[query] = {doc: {'r': float(rank), 'f': np.array(feature)}}
        return query_doc_feature

## Generator

Here we need to use `tf.variable_scope` for two reasons. Firstly, we're going to make sure all the variable names start with `generator`. Similarly, we'll prepend `discriminator` to the discriminator variables. This will help out later when we're training the separate networks.

Here's more from [the TensorFlow documentation](https://www.tensorflow.org/programmers_guide/variable_scope#the_problem) to get another look at using `tf.variable_scope`.

In [3]:
class Generator:
    def __init__(self, feature_size, hidden_size, keep_prob=1.0):
        self.feature_size = feature_size
        self.hidden_size = hidden_size
        self.keep_prob = keep_prob
        
        with tf.variable_scope('generator'):
            # input placeholders
            self.reward = tf.placeholder(tf.float32, [None], name='reward')
            self.pred_data = tf.placeholder(tf.float32, [None, self.feature_size], name='pred_data')
            self.sample_index = tf.placeholder(tf.int32, [None], name='sample_index')
            
            ########## score of RankNet ##########

            # trainable variables
            self.weight_1 = tf.Variable(tf.truncated_normal([self.feature_size, self.hidden_size], mean=0.0, stddev=0.1), name='weight_1')
            self.bias_1 = tf.Variable(tf.zeros([self.hidden_size]), name='bias_1')
            self.weight_2 = tf.Variable(tf.truncated_normal([self.hidden_size, 1], mean=0.0, stddev=0.1), name='weight_2')
            self.bias_2 = tf.Variable(tf.zeros([1]), name='bias_2')

            # layer 1 (hidden layer)
            self.layer_1 = tf.nn.tanh(tf.nn.xw_plus_b(self.pred_data, self.weight_1, self.bias_1))
            
            # dropout
            self.dropout_1 = tf.nn.dropout(self.layer_1, self.keep_prob)

            # layer 2 (output layer)
            self.layer_2 = tf.nn.xw_plus_b(self.dropout_1, self.weight_2, self.bias_2)
            
            #################################
            
            # probability distribution
            self.opt_prob =tf.gather(tf.reshape(tf.nn.softmax(tf.reshape(self.layer_2, [1, -1])), [-1]), self.sample_index)
            
            # loss for optimization
            self.opt_loss = -tf.reduce_mean(tf.log(self.opt_prob) * self.reward) # minus signe is needed for maximum.
            
            # score for prediction
            self.pred_score = tf.nn.xw_plus_b(self.layer_1, self.weight_2, self.bias_2)
            self.pred_score = tf.reshape(self.pred_score, [-1])

## Discriminator

This is a pairwaise case implimentation.

In [4]:
class Discriminator:
    def __init__(self, feature_size, hidden_size, keep_prob=1.0):
        self.feature_size = feature_size
        self.hidden_size = hidden_size
        self.keep_prob = keep_prob
        
        with tf.variable_scope('discriminator'):
            # input placeholders
            self.pos_data = tf.placeholder(tf.float32, [None, self.feature_size], name='pos_data')
            self.neg_data = tf.placeholder(tf.float32, [None, self.feature_size], name='neg_data')
            self.pred_data = tf.placeholder(tf.float32, [None, self.feature_size], name='pred_data')

            ########## score of RankNet ##########
            
            ## trainable variables
            self.weight_1 = tf.Variable(tf.truncated_normal([self.feature_size, self.hidden_size], mean=0.0, stddev=0.1), name='weight_1')
            self.bias_1 = tf.Variable(tf.zeros([self.hidden_size]), name='bias_1')
            self.weight_2 = tf.Variable(tf.truncated_normal([self.hidden_size, 1], mean=0.0, stddev=0.1), name='weight_2')
            self.bias_2 = tf.Variable(tf.zeros([1]), name='bias_2')
            
            # layer 1 (hidden layer)
            self.pos_layer_1 = tf.nn.tanh(tf.nn.xw_plus_b(self.pos_data, self.weight_1, self.bias_1))
            self.neg_layer_1 = tf.nn.tanh(tf.nn.xw_plus_b(self.neg_data, self.weight_1, self.bias_1))
            
            # dropout
            self.pos_dropout_1 = tf.nn.dropout(self.pos_layer_1, self.keep_prob)
            self.neg_dropout_1 = tf.nn.dropout(self.neg_layer_1, self.keep_prob)
            
            # layer 2 (output layer)
            self.pos_layer_2 = tf.nn.xw_plus_b(self.pos_dropout_1, self.weight_2, self.bias_2)
            self.neg_layer_2 = tf.nn.xw_plus_b(self.neg_dropout_1, self.weight_2, self.bias_2)
            
            #################################
            
            # loss for optimization
            self.opt_loss = -tf.reduce_mean(tf.log(tf.sigmoid(self.pos_layer_2 - self.neg_layer_2))) # minus signe is needed for miximum
            
            # reward for generator
            self.reward = tf.reshape(tf.log(1 + tf.exp(self.neg_layer_2 - self.pos_layer_2)), [-1])
            
            # score for prediction
            self.pred_score = tf.nn.xw_plus_b(tf.nn.tanh(tf.nn.xw_plus_b(self.pred_data, self.weight_1, self.bias_1)), self.weight_2, self.bias_2)
            self.pred_score = tf.reshape(self.pred_score , [-1])

## Optimizer

We want to update the generator and discriminator variables separately. So we need to get the variables for each part build optimizers for the two parts. To get all the trainable variables, we use `tf.trainable_variables()`. This creates a list of all the variables we've defined in our graph.

For the generator optimizer, we only want to generator variables. Our past selves were nice and used a variable scope to start all of our generator variable names with `generator`. So, we just need to iterate through the list from `tf.trainable_variables()` and keep variables to start with `generator`. Each variable object has an attribute `name` which holds the name of the variable as a string (`var.name == 'weights_0'` for instance). 

We can do something similar with the discriminator. All the variables in the discriminator start with `discriminator`.

Then, in the optimizer we pass the variable lists to `var_list` in the `minimize` method. This tells the optimizer to only update the listed variables. Something like `tf.train.AdamOptimizer().minimize(loss, var_list=var_list)` will only train the variables in `var_list`.

In [5]:
class Optimizer:
    def __init__(self, g, d, learning_rate):
        # get the trainable_variables, split into generator and discriminator parts.
        t_vars = tf.trainable_variables()
        self.g_vars = [var for var in t_vars if var.name.startswith('generator')]
        self.d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

        self.g_train_opt = tf.train.AdamOptimizer(learning_rate).minimize(g.opt_loss, var_list=self.g_vars)        
        self.d_train_opt = tf.train.AdamOptimizer(learning_rate).minimize(d.opt_loss, var_list=self.d_vars)

## Dataset (MQ2008-semi extension)

If there are only two levels of relevance and for each "observed" relevant-irrelevant document pair (d_i, d_j) we sample an unlabelled document d_k to form the "generated" document pair (d_k, d_j), then it can be shown that the objective function of the IRGAN-pairwise minimax game Eq. (7) in the paper is bounded by the mathematical expectation of (f_phi(d_i, q) - f_phi(d_k, q)) / 2 which is independent of the irrelevant document d_j, via a straightforward application of Jensen's inequality on the logarithm function.

In [6]:
class Dataset(MQ2008):
    def __init__(self, batch_size, dataset_dir='MQ2008-semi'):
        MQ2008.__init__(self, dataset_dir=dataset_dir)
        self.batch_size = batch_size
        self.docs_pairs = []
    
    def set_docs_pairs(self, sess, generator):
        for query in dataset.get_pos_queries():
            can_docs = dataset.get_docs(query)
            can_features = [dataset.get_features(query, doc) for doc in can_docs]
            can_score = sess.run(generator.pred_score, feed_dict={generator.pred_data: can_features})
        
            # softmax for candidate
            exp_rating = np.exp(can_score)
            prob = exp_rating / np.sum(exp_rating)
        
            pos_docs = dataset.get_pos_docs(query)
            neg_docs = []
            for i in range(len(pos_docs)):
                while True:
                    doc = np.random.choice(can_docs, p=prob)
                    if doc not in pos_docs:
                        neg_docs.append(doc)
                        break
            
            for i in range(len(pos_docs)):
                self.docs_pairs.append((query, pos_docs[i], neg_docs[i]))
        
    def get_batches(self):
        size = len(self.docs_pairs)
        cut_off = size // self.batch_size
        
        for i in range(0, self.batch_size * cut_off, self.batch_size):
            batch_pairs = self.docs_pairs[i:i+self.batch_size]
            yield np.asarray([self.get_features(p[0], p[1]) for p in batch_pairs]), np.asarray([self.get_features(p[0], p[2]) for p in batch_pairs])

## Training

There are some auxiliary functions.

In [7]:
def train_generator(sess, generator, discriminator, optimizer, dataset):
    for query in dataset.get_pos_queries():
        pos_docs = dataset.get_pos_docs(query)
        can_docs = dataset.get_docs(query)
        
        can_features = [dataset.get_features(query, doc) for doc in can_docs]
        can_score = sess.run(generator.pred_score, feed_dict={generator.pred_data: can_features})
        
        # softmax for all
        exp_rating = np.exp(can_score)
        prob = exp_rating / np.sum(exp_rating)
        
        # sampling
        neg_index = np.random.choice(np.arange(len(can_docs)), size=[len(pos_docs)], p=prob)
        neg_docs = np.array(can_docs)[neg_index]
        
        pos_features =  [dataset.get_features(query, doc) for doc in pos_docs]
        neg_features = [dataset.get_features(query, doc) for doc in neg_docs]
        
        neg_reward = sess.run(discriminator.reward,
                              feed_dict={discriminator.pos_data: pos_features, discriminator.neg_data: neg_features})
            
        _ = sess.run(optimizer.g_train_opt, 
                     feed_dict={generator.pred_data: can_features, generator.sample_index: neg_index, generator.reward: neg_reward})
            
    return sess.run(generator.opt_loss, 
                    feed_dict={generator.pred_data: can_features, generator.sample_index: neg_index, generator.reward: neg_reward})

In [8]:
def train_discriminator(sess, generator, discriminator, optimizer, dataset):
    dataset.set_docs_pairs(sess, generator)
    
    for input_pos, input_neg in dataset.get_batches():
        _ = sess.run(optimizer.d_train_opt,
                     feed_dict={discriminator.pos_data: input_pos, discriminator.neg_data: input_neg})
        
    return sess.run(discriminator.opt_loss, 
                    feed_dict={discriminator.pos_data: input_pos, discriminator.neg_data: input_neg})

In [9]:
def ndcg_at_k(sess, discriminator, dataset, k=5):
    ndcg = 0.0
    cnt = 0

    for query in dataset.get_pos_queries(target='test'):
        pos_docs = dataset.get_pos_docs(query, target='test')
        pred_docs = dataset.get_docs(query, target='test')

        if len(pred_docs) < k:
            continue

        pred_features = np.asarray([dataset.get_features(query, doc, target='test') for doc in pred_docs])
        pred_score = sess.run(discriminator.pred_score, feed_dict={discriminator.pred_data: pred_features})
        pred_doc_score = sorted(zip(pred_docs, pred_score), key=lambda x: x[1], reverse=True)
        
        dcg = 0.0
        for i in range(0, k):
            doc, _ = pred_doc_score[i]
            if doc in pos_docs:
                dcg += (1 / np.log2(i + 2))

        n = len(pos_docs) if len(pos_docs) < k else k
        idcg = np.sum(np.ones(n) / np.log2(np.arange(2, n + 2)))

        ndcg += (dcg / idcg)
        cnt += 1

    return ndcg / float(cnt)

There are hyperparameters.

In [10]:
# size of input vector
feature_size = 46
# size of latent vector
hidden_size = 46
# keep probability
keep_prod = 0.5
# learning_rate
learning_rate = 0.00001
# batch_size
batch_size = 8
# generator training epochs
epochs = 30

Some objects are created.

In [11]:
dataset = Dataset(batch_size)

In [12]:
generator = Generator(feature_size, hidden_size, keep_prod)
discriminator = Discriminator(feature_size, hidden_size, keep_prod)
optimizer = Optimizer(generator, discriminator, learning_rate)

/Users/takahish/anaconda/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


A generator and discriminator are optimized with dataset.

In [13]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for e in range(epochs):
        g_loss = train_generator(sess, generator, discriminator, optimizer, dataset)
        d_loss = train_discriminator(sess, generator, discriminator, optimizer, dataset)
        
        ndcg_at_3 = ndcg_at_k(sess, discriminator, dataset, k=3)
        ndcg_at_5 = ndcg_at_k(sess, discriminator, dataset, k=5)
        ndcg_at_10 = ndcg_at_k(sess, discriminator, dataset, k=10)
        
        print("Epoch {}/{}...".format(e+1, epochs),
              "Generator Loss: {:.4f}".format(g_loss), 
              "Discriminator Loss: {:.4f}".format(d_loss),
              "NDCG@3: {:.4f}".format(ndcg_at_3),
              "NDCG@5: {:.4f}".format(ndcg_at_5),
              "NDCG@10: {:.4f}".format(ndcg_at_10))                                   

Epoch 1/30... Generator Loss: 4.9770 Discriminator Loss: 0.6435 NDCG@3: 0.0174 NDCG@5: 0.0187 NDCG@10: 0.0249
Epoch 2/30... Generator Loss: 4.8975 Discriminator Loss: 0.7134 NDCG@3: 0.0485 NDCG@5: 0.0493 NDCG@10: 0.0612
Epoch 3/30... Generator Loss: 5.2394 Discriminator Loss: 0.7249 NDCG@3: 0.1158 NDCG@5: 0.1179 NDCG@10: 0.1270
Epoch 4/30... Generator Loss: 4.9916 Discriminator Loss: 0.7441 NDCG@3: 0.1305 NDCG@5: 0.1262 NDCG@10: 0.1355
Epoch 5/30... Generator Loss: 5.0673 Discriminator Loss: 0.6937 NDCG@3: 0.1282 NDCG@5: 0.1285 NDCG@10: 0.1386
Epoch 6/30... Generator Loss: 5.4678 Discriminator Loss: 0.7267 NDCG@3: 0.1271 NDCG@5: 0.1316 NDCG@10: 0.1469
Epoch 7/30... Generator Loss: 5.6698 Discriminator Loss: 0.6418 NDCG@3: 0.1315 NDCG@5: 0.1337 NDCG@10: 0.1524
Epoch 8/30... Generator Loss: 4.9896 Discriminator Loss: 0.5203 NDCG@3: 0.1341 NDCG@5: 0.1356 NDCG@10: 0.1580
Epoch 9/30... Generator Loss: 4.4342 Discriminator Loss: 0.6565 NDCG@3: 0.1494 NDCG@5: 0.1456 NDCG@10: 0.1648
Epoch 10/3